# Hello World in `srun`

Below is the absolute simplest way to run a job on Cheaha.

In [ ]:
!srun --pty echo "Hello World!"

While `srun` is great for setting up an interactive job using `--pty`, it doesn't scale well to repeated use or for intricate jobs. It's hard to remember, share and modify. A job created with `srun` will also end early, possibly destroying your work, if you lose your connection to Cheaha!

Prefer using `sbatch` over `srun`. With `sbatch` you write a script with one resource request per line, and can sequence multiple tasks, all in one place. These tasks can take the form of trackable job steps using `srun`, or just using plain shell commands. It is also possible to use job arrays to submit many similar jobs at the same time. Using `sbatch` makes it easy to

- share with collaborators
- keep track of versions -- repeatability!
- read and modify
- run multiple commands in one job
- run many of the same type of job with a single submission
- won't terminate if you lose connection

Remember, `srun` is only useful for interactive jobs, one-off commands, and sub-tasks inside an `sbatch` job. In contrast `sbatch` is meant for repeatable, collaborative Research Computing!

Let's take a look at how to use `sbatch`.

# Hello World in `sbatch`


Here is a sample script which waits a few seconds after submission, then `echo`s a couple of lines to an output file.

The cell below uses the `ipython` magic `%%bash` to run the contents of the cell using the `bash` shell, as though typed at a terminal. The `cat` command concatenates things to a stream. The `> "hello_world.sh"` means we are redirecting the output of `cat` to the file `hello_world.sh`. The `<<EOT` starts a `heredoc` and redirects it into `cat`. A `heredoc` is what it sounds like, a "fake" file we are making up as we go. Our heredoc starts on the line after `cat` and ends on the line before `EOT`.

Basically, we're writing what you see into a file that we can use later. When you write a script on your own later, you won't need to do this. Instead, just open your favorite text editor and start with the line `#! /bin/bash` and up to, but not including, `EOT` at the end. We have the extra parts here so you can see the contents of the file directly in Jupyter without having to open the file.

In [ ]:
%%bash
cat <<EOT > "hello_world.sh"
#! /bin/bash

## BOOKKEEPING
#SBATCH --job-name=hello_world
# %x means "put the job-name here"
#SBATCH --output=%x.log
#SBATCH --error=%x.log

## RESOURCES
#SBATCH --partition=express
#SBATCH --time=00:01:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=256M

# PAYLOAD
sleep 15
echo "hello world"
echo "hi again"
EOT

We can verify that the contents of `hello_world.sh` are what we expect using `cat` again. The `!` symbol is `ipython` magic that runs the line as a shell command. Make sure the output matches the cell above!

So `%%bash` runs an entire cell in `bash` and `!` runs a single line in `bash`.

In [ ]:
!cat "hello_world.sh"

Now that we are sure our `hello_world.sh` file is prepared, we can submit the script to the slurm queue using sbatch.

In [ ]:
!sbatch "hello_world.sh"

We can check that the job made it into the queue using `squeue`. You'll want to do this quickly because the job will only be around for about 15 seconds (see the line with `sleep 15`). While we're at it, please make a note of this job id number for later in the session.

You should also see the `ood-jupyter` job you are using to run these cells as well!

In [ ]:
!squeue -u $USER

We can verify the output of the script using `cat` again.

In [ ]:
!cat "hello_world.log"

Congratulations, you now know the workflow for using `sbatch` scripts! There are a couple more commands that you may find useful. Before we go any further, let's first talk about some good practices.

Taken from the [Zen of Python](https://www.python.org/dev/peps/pep-0020/), we see that "explicit is better than implicit." So, for `sbatch` scripts, don't rely on default values! Instead, be explicit about your intent for the job submission. That way other people, or you at a later date, will be able to understand what you meant. Always...

- ...give your jobs meaningful names with `--job-name`
- ...specify your output logs with `--output` and `--error`
- ...choose `--partition` and cpu, memory, and gpu resources carefully

Now that we've seen some good practices, let's see what happens if you submit a job by mistake and need to cancel it to free up resources.

# How to Cancel a Job
Below we'll run the exact same workflow as we did for `hello_world.sh`. We'll call this one `cancelme.sh` because we're going to learn how to cancel a job. We accidentally made the job sleep for 3600 seconds (1 hour) before getting to the good part. We meant to only wait 15 seconds, but we didn't realize it until after we submitted the job. Oops!

Let's run through the same steps we did above...

Write to a file...

In [ ]:
%%bash --out t
cat <<EOT > "cancelme.sh"
#! /bin/bash

## BOOKKEEPING
#SBATCH --job-name=cancelme
# %x means "put the job-name here"
#SBATCH --output=%x.log
#SBATCH --error=%x.log

## RESOURCES
#SBATCH --partition=express
#SBATCH --time=01:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=256M

# PAYLOAD
sleep 3600  ## Oops we meant 15 seconds!
echo "finally!"
EOT

Check the file...

In [ ]:
!cat "cancelme.sh"

Submit the job...

In [ ]:
!sbatch cancelme.sh

Check the queue...

In [ ]:
!squeue -u $USER

Our `cancelme` job is going to take an hour to clear from the queue, meanwhile we're just using up shared resources and making it harder for us to get our jobs run, due to reduced priority! Let's be good HPC citizens and cancel that mistaken job.

To cancel the job you'll need to edit the cell below. Where you see `[jobid]` below, replace it with the job id from our `cancelme.sh` submission. Think about the cell where you ran `!sbatch cancelme.sh`...

In [ ]:
!scancel [jobid]

Let's check that we actually canceled the job. We can do that by making sure it's not in the queue any longer.

In [ ]:
!squeue -u $USER

# How to Review Past Jobs

Here we are going to see how to review previous jobs using the `sacct` command. This command can be useful when you need support or for diagnosing SLURM issues related to your jobs, or just for better understanding your own usage history.

First, let's try just looking at one job. Grab that job id number you saved from much earlier, you'll need it for this and the next section. Replace `[jobid]` below with the job id number from earlier and run the cell. The `-j` or `--jobs` flag indicates you want to look at one or more jobs. Also try it with a comma-separated list like `[jobid1],[jobid2]` using the two previous job ids.

In [ ]:
!sacct -j [jobid]

We can also review past jobs based on a start date using the `-S` or `--startdate` flag. Enter today's date in the format `YYYY-MM-DD` where it says `[date]` below.

In [ ]:
!sacct -S [date]

By default only jobs starting on or after the start of the specified date, local time, will be shown. You can use a more specific time using an ISO-like format, e.g. 2021-09-08T13:15:00. You can also use the `-E` or `--enddate` flag to set an end date. In this case it would be only jobs before the start of the specified date, local time.

Another useful flag is `-o` or `--format`. There are many options for output fields available [here](https://slurm.schedmd.com/sacct.html#lbAF). The fields can be strung together using a comma-separated list. We won't go into detail on them, but feel free to give them a try later. Replace `[jobid]` in the cell below and run it to get a feel for how it can work. Here we've added `alloccpu` to see how many CPUs and `reqmem` to see how much memory were allocated to the job.

In [ ]:
!sacct -j [jobid] -o jobid,start,end,state,alloccpu,reqmem

# How to Increase Efficiency

When using shared resources we of course want to be as efficient as possible. It helps us by increasing our job priorities. It helps us by allowing us to fit our jobs in sooner. It helps others by leaving as much room available as possible. It's part of being a good HPC citizen.

An important tool that can help understand resource usage efficiency is the `seff` command. Grab that job id number you saved from much earlier, again, and replace `[jobid]` in the next cell and running it.

In [ ]:
!seff [jobid]

A neat summary of job statistics will appear which gives insight into the job's efficiency.

For the "Hello World!" job, of course the efficiency will be pretty low, because we are using hardly any resources to print a string to the terminal. It shouldn't be more than a few megabytes. What happens if we re-run the first job with `32M` instead of `256M` memory?

# Hello World with Arrays!

Sometimes we want to run the same analysis on multiple distinct, but similarly-shaped, data sets. We call this of analysis pleasingly parallel. Others may call it embarrassingly parallel. Rather than writing or modifying an `sbatch` script for each data set, which can be tedious and error-prone, we can write one `sbatch` script and use the `--array` flag.

The script below has four differences. The first is the `--job-name`, we added `_array` to the end so we don't overwrite the old log file. The second is to change `%x.log` to `%x_%a.log`, which will append the task number to the log name. That way each task writes to its own unique log file.

The third change is the `## ARRAY` section between `## RESOURCES` and `# PAYLOAD`. Any comma and hyphen separated list of non-negative integers can be used here. The list tells `sbatch` which specific tasks to run. The value after the `%` symbol tells `sbatch` the largest number of tasks we want running at one time.

The fourth change is to the `# PAYLOAD`. SLURM creates certain bash variables (more info [here](https://slurm.schedmd.com/sbatch.html#lbAK)) that are available inside the job context. The most useful one here is the `SLURM_ARRAY_TASK_ID` which, for each task, has the task number from the list given to the `--array` flag. So for task 1 this would have the value `1`, etc.

In [ ]:
%%bash
cat <<EOT > "hello_world.sh"
#! /bin/bash

## BOOKKEEPING
#SBATCH --job-name=hello_world_array
# %x means "put the job-name here"
# %a means "put the array task id here"
#SBATCH --output=%x_%a.log
#SBATCH --error=%x_%a.log

## RESOURCES
#SBATCH --partition=express
#SBATCH --time=00:01:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=256M

## ARRAY
#SBATCH --array=1-4,10-12%4

# PAYLOAD
echo "hello world from #\$SLURM_ARRAY_TASK_ID"
EOT

In [ ]:
!sbatch "hello_world.sh"

When we run `squeue` we will notice something different. Specifically, that there are jobs like `#_1` or `#_10`. These correspond to each individual task in the `--array`.

In [ ]:
!squeue -u $USER

Once the tasks are all complete, try replacing `[number]` in the file below with different task id numbers from the `--array` flag.

In [ ]:
!cat "hello_world_array_[number].log"

The `SLURM_ARRAY_TASK_ID` variable can be used in clever ways to interface with other languages, like MATLAB and Python, and prepare parameters dynamically. One way to have dynamic parameters is to create a CSV file where each row has the list of parameters for that task id number, then read that line from the CSV file. Setting up interfaces is specific to each research computing task and may take some experimentation.

# How to Request GPUs

Some research projects are greatly facilitated by using GPUs. Machine learning and deep learning tasks often experience speed ups close to 50 to 100 times when using a modern GPU compared to a modern CPU. To request a GPU we can use the `--gres` flag. The following script will request a GPU using `--gres=gpu:1`, `module load` the necessary CUDA drivers, and poll the current state of the GPU with `nvidia-smi`.

In [ ]:
%%bash
cat <<EOT > "hello_world_gpu.sh"
#! /bin/bash

## BOOKKEEPING
#SBATCH --job-name=hello_world_gpu
# %x means "put the job-name here"
#SBATCH --output=%x.log
#SBATCH --error=%x.log

## RESOURCES
#SBATCH --partition=express
#SBATCH --time=00:01:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=2G
#SBATCH --gres=gpu:1

# PAYLOAD
module load cuda11.3/toolkit/11.3.0
nvidia-smi
EOT

In [ ]:
!sbatch "hello_world_gpu.sh"

In [ ]:
!squeue -u $USER

Once the job is complete, check the output by running the cell below. You should see a timestamp and then a table containing information about the GPU's current state, including temperature, power usage, memory usage, utilization, and driver and CUDA versions.

In [ ]:
!cat "hello_world_gpu.log"

# Conclusions and Next Steps